# Modify json for our application


In [93]:
pip install azure

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: isodate>=0.6.0 in c:\users\django\appdata\local\programs\python\python310\lib\site-packages (from msrest<2.0.0,>=0.5.4->azure-applicationinsights~=0.1.0->azure) (0.6.0)



You should consider upgrading via the 'C:\Users\Django\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [94]:
pip install azure-cognitiveservices-language-luis

You should consider upgrading via the 'C:\Users\Django\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:

from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from functools import reduce

import json, time, uuid

In [4]:
import json
import pandas as pd

In [97]:
df=pd.read_json('../frames/frames.json')

df

,user_id,turns,wizard_id,id,labels
0,U22HTHYNP,[{'text': 'I'd like to book a trip to Atlantis...,U21DKG18C,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."
1,U21E41CQP,"[{'text': 'Hello, I am looking to book a vacat...",U21DMV0KA,4a3bfa39-2c22-42c8-8694-32b4e34415e9,"{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc..."
2,U21RP4FCY,[{'text': 'Hello there i am looking to go on a...,U21E0179B,6e67ed28-e94c-4fab-96b6-68569a92682f,"{'userSurveyRating': 2.0, 'wizardSurveyTaskSuc..."
3,U22HTHYNP,[{'text': 'Hi I'd like to go to Caprica from B...,U21DKG18C,5ae76e50-5b48-4166-9f6d-67aaabd7bcaa,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
4,U21E41CQP,"[{'text': 'Hello, I am looking to book a trip ...",U21DMV0KA,24603086-bb53-431e-a0d8-1dcc63518ba9,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
...,...,...,...,...,...
1364,U2AMZ8TLK,[{'text': 'Hi I've got 9 days free and I'm loo...,U21DMV0KA,957fd205-bb7c-4b81-8cb6-13c81c51c5c9,"{'userSurveyRating': 3.5, 'wizardSurveyTaskSuc..."
1365,U2AMZ8TLK,[{'text': 'I need to get to Fortaleza on Septe...,U260BGVS6,71b21b86-2d05-4372-a0ee-6ed64b0ddc42,"{'userSurveyRating': 4.5, 'wizardSurveyTaskSuc..."
1366,U231PNNA3,[{'text': 'We're finally going on vacation isn...,U21T9NMKM,ef2cd70e-c1f2-42be-8839-cb465af0bf41,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
1367,U2AMZ8TLK,"[{'text': 'Hi there, I'm looking for a place t...",U21DMV0KA,ffa79d2c-14eb-45e6-8573-b0817a1a1803,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."


In [98]:
len(df)

1369

In [99]:
x=df.iloc[0].turns[0]
x

{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
 'labels': {'acts': [{'args': [{'val': 'book', 'key': 'intent'}],
    'name': 'inform'},
   {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
     {'val': 'Caprica', 'key': 'or_city'},
     {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
     {'val': '8', 'key': 'n_adults'},
     {'val': '1700', 'key': 'budget'}],
    'name': 'inform'}],
  'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
    'name': 'inform'},
   {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
     {'val': 'Caprica', 'key': 'or_city'},
     {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
     {'val': '8', 'key': 'n_adults'},
     {'val': '1700', 'key': 'budget'}],
    'name': 'inform'}],
  'active_frame': 1,
  'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
     'budget': [{'val': '1700.0', 'negated': False}],
     'dst_city': [{

In [100]:
x['labels']['frames'][0]['info']['dst_city'][0]['val']

'Atlantis'

In [101]:
df.loc[0].turns[0]['labels']['frames'][0]['info']['dst_city'][0]['val']

'Atlantis'

In [102]:
data=[]
dst_city='nan'
or_city='nan'
budget='0'
str_date='nan'
end_date='nan'
for i in range(0,1367):
    where=df.loc[i].turns[0]
    text=where['text']
    if 'dst_city' in where['labels']['frames'][0]['info']:
        dst_city= where['labels']['frames'][0]['info']['dst_city'][0]['val']
        if dst_city== '-1':
            dst_city='nan'
    if 'or_city' in where['labels']['frames'][0]['info']:
        or_city= where['labels']['frames'][0]['info']['or_city'][0]['val']
        if or_city== '-1':
            or_city='nan'

    if 'budget' in where['labels']['frames'][0]['info']:
        budget= where['labels']['frames'][0]['info']['budget'][0]['val']
        
        if budget== '-1':
            budget='0'
        elif budget== '$MIN':
            budget='0'
        elif budget== 'free':  #WE CLEAN EVERY EXCEPTIONS
            budget='0'
        elif budget== 'travel for free':
            budget='0'
        elif budget== 'all my savings':
            budget='0'                
            
    if 'str_date' in where['labels']['frames'][0]['info']:
        str_date= where['labels']['frames'][0]['info']['str_date'][0]['val']
        if str_date== '-1':
            str_date='nan'
    if 'end_date' in where['labels']['frames'][0]['info']:
        end_date=where['labels']['frames'][0]['info']['end_date'][0]['val']
        if end_date== '-1':
            end_date='nan'
    data.append([text,dst_city,or_city,budget,str_date,end_date])
dataframe=pd.DataFrame(data, columns=["text","dst_city","or_city","budget","str_date","end_date"])

    
    
        
        
         



- We have now 1000 json almost clean

In [103]:
dataframe

,text,dst_city,or_city,budget,str_date,end_date
0,I'd like to book a trip to Atlantis from Capri...,Atlantis,Caprica,1700.0,august 13,nan
1,"Hello, I am looking to book a vacation from Go...",Mos Eisley,Gotham City,2100.0,august 13,nan
2,Hello there i am looking to go on a vacation w...,Gotham City,Gotham City,2100.0,august 13,nan
3,"Hi I'd like to go to Caprica from Busan, betwe...",Caprica,Busan,2100.0,august 21,august 31
4,"Hello, I am looking to book a trip for 2 adult...",Denver,Kochi,21.3,august 21,august 31
...,...,...,...,...,...,...
1362,"Hey, I need to get back home to Milan!",Milan,Recife,4000.0,september 30,4
1363,I have a CRUCIAL meeting with my investors in ...,Santa Cruz,Recife,4000.0,september 30,4
1364,Hi I've got 9 days free and I'm looking for a ...,Santa Cruz,San Francisco,4000.0,september 30,4
1365,I need to get to Fortaleza on September 8th or...,Fortaleza,San Francisco,4000.0,september 8,4


- We need to convert some budgets

In [104]:
def convert(budget):
    if budget<100:
        budget=budget*1000
    return budget

In [105]:
dataframe['budget']=dataframe['budget'].astype(str).astype(float)
dataframe['budget']=dataframe['budget'].transform(convert)
dataframe['budget']=dataframe['budget'].astype(int).astype(str)
# Lower every character
dataframe['text']=dataframe['text'].str.lower()
dataframe['dst_city']=dataframe['dst_city'].str.lower()
dataframe['or_city']=dataframe['or_city'].str.lower()

In [106]:
dataframe

,text,dst_city,or_city,budget,str_date,end_date
0,i'd like to book a trip to atlantis from capri...,atlantis,caprica,1700,august 13,nan
1,"hello, i am looking to book a vacation from go...",mos eisley,gotham city,2100,august 13,nan
2,hello there i am looking to go on a vacation w...,gotham city,gotham city,2100,august 13,nan
3,"hi i'd like to go to caprica from busan, betwe...",caprica,busan,2100,august 21,august 31
4,"hello, i am looking to book a trip for 2 adult...",denver,kochi,21300,august 21,august 31
...,...,...,...,...,...,...
1362,"hey, i need to get back home to milan!",milan,recife,4000,september 30,4
1363,i have a crucial meeting with my investors in ...,santa cruz,recife,4000,september 30,4
1364,hi i've got 9 days free and i'm looking for a ...,santa cruz,san francisco,4000,september 30,4
1365,i need to get to fortaleza on september 8th or...,fortaleza,san francisco,4000,september 8,4


Now that we have a pretty clean dataframe we can start to create a new json.

We need our structure to be compatible with LUIS

## JSON For LUIS

In [107]:
intentName="FlyMe"

In [108]:
def Json_Luis(data):
    loc=[]
    where=['or_city','dst_city','budget','str_date','end_date']
    for arg in where:
        where_arg=data[arg]
        #print(arg,where_arg)
        if (where_arg!='nan'):
            st=data.text.find(where_arg) #Find location 
            if(st==-1 or where_arg=='0'): # In case there is no location returned
                loc.append([0,0])
            else:
                ed=st+len(where_arg)-1 #End Char Index
                loc.append([st,ed])
        else :loc.append([0,0])
        #print(loc)
        #print(st,ed)
    
    labeledExample={
    "text": data.text,
    "intentName": intentName,
    "entityLabels": [
        {
            "startCharIndex": 0,
            "endCharIndex": (len(data.text)-1),
            "entityName": "FlyMeBooking",
            "children": [
                {
                    "startCharIndex": loc[0][0],
                    "endCharIndex": loc[0][1],
                    "entityName": "or_city"
                    
                },
                
                {
                    "startCharIndex": loc[1][0],
                    "endCharIndex": loc[1][1],
                    "entityName": "dst_city"
                },
                {
                    "startCharIndex": loc[2][0],
                    "endCharIndex": loc[2][1],
                    "entityName": "budget"
                },
                {
                    "startCharIndex": loc[3][0],
                    "endCharIndex": loc[3][1],
                    "entityName": "str_date"
                },
                {
                    "startCharIndex": loc[4][0],
                    "endCharIndex": loc[4][1],
                    "entityName": "end_date"
                }                   
                        
            ]}
        
        ]}
    
        # Remove all the empty childrens in the json
    removeEmpty=[]
    for index, localisation, in enumerate(loc):
        if (localisation==[0,0]):
            #print('i am in')
            removeEmpty.append(labeledExample['entityLabels'][0]['children'][index]['entityName'])
    #print(labeledExample['entityLabels'][0]['children'][index]['entityName'])
    #print(removeEmpty)
    for rm in removeEmpty:
        for i in range(len(labeledExample['entityLabels'][0]['children'])):
            
            if labeledExample['entityLabels'][0]['children'][i]['entityName']==rm:
    
                labeledExample['entityLabels'][0]['children'].pop(i)
                break

    return labeledExample              

In [109]:
Json_Luis(dataframe.iloc[0])

{'text': "i'd like to book a trip to atlantis from caprica on saturday, august 13, 2016 for 8 adults. i have a tight budget of 1700.",
 'intentName': 'FlyMe',
 'entityLabels': [{'startCharIndex': 0,
   'endCharIndex': 121,
   'entityName': 'FlyMeBooking',
   'children': [{'startCharIndex': 41,
     'endCharIndex': 47,
     'entityName': 'or_city'},
    {'startCharIndex': 27, 'endCharIndex': 34, 'entityName': 'dst_city'},
    {'startCharIndex': 117, 'endCharIndex': 120, 'entityName': 'budget'},
    {'startCharIndex': 62, 'endCharIndex': 70, 'entityName': 'str_date'}]}]}

Works fine the last children has been deleted cause there is no end date

# Luis Training
## Authentification 

In [110]:
def get_grandchild_id(model, childName, grandChildName):
    
    theseChildren = next(filter((lambda child: child.name == childName), model.children))
    theseGrandchildren = next(filter((lambda child: child.name == grandChildName), theseChildren.children))
    
    grandChildId = theseGrandchildren.id
    
    return grandChildId


In [111]:
def quickstart():
    
    # add calls here, remember to indent properly

    quickstart()

In [5]:
authoringKey = '2cde43969cde4c70b1612e330e0cadcd'
authoringEndpoint = 'https://lasta-authoring.cognitiveservices.azure.com/'
predictionKey = '6b8d76e7fde4489cb8c8a2181b365923'
predictionEndpoint = 'https://lasta.cognitiveservices.azure.com/'
# We use a UUID to avoid name collisions.
appName = "Fly Me Booking Alpha " + str(uuid.uuid4())
versionId = "1"


client = LUISAuthoringClient(authoringEndpoint, CognitiveServicesCredentials(authoringKey))

In [113]:
# define app basics
appDefinition = ApplicationCreateObject (name=appName, initial_version_id=versionId, culture='en-us')

# create app
app_id = client.apps.add(appDefinition)

# get app id - necessary for all other changes
print("Created LUIS app with ID {}".format(app_id))

Created LUIS app with ID c4cb4b1a-f70d-4575-a4d5-b04c7e1d4812


In [114]:
client.model.add_intent(app_id, versionId, intentName)

# Add Prebuilt entity
client.model.add_prebuilt(app_id, versionId, prebuilt_extractor_names=["number"])

In [115]:
# define machine-learned entity
mlEntityDefinition =[

        { "name": "or_city" },
        { "name": "dst_city"},
        { "name": "budget" },
        {"name" :"str_date"},
        {"name" :"end_date"}
    ]


In [116]:
# add entity to app
modelId = client.model.add_entity(app_id, versionId, name="FlyMeBooking", children=mlEntityDefinition)

In [117]:
modelId

'481b7b5b-35e0-48e9-8767-9c2d1e655070'

## Add Examples to Train Luis

In [118]:
from time import sleep

In [119]:
for i in range(0,1100): #80% Train 20% Test
    labeledExampleUtteranceWithMLEntity=Json_Luis(dataframe.iloc[i])
    
    try:
        client.examples.add(app_id, versionId, labeledExampleUtteranceWithMLEntity)
    except:
        print("The id doesn't work :",i)

The id doesn't work : 2
The id doesn't work : 19
The id doesn't work : 21
The id doesn't work : 32
The id doesn't work : 47
The id doesn't work : 129
The id doesn't work : 182
The id doesn't work : 265
The id doesn't work : 310
The id doesn't work : 382
The id doesn't work : 389
The id doesn't work : 425
The id doesn't work : 429
The id doesn't work : 430
The id doesn't work : 456
The id doesn't work : 457
The id doesn't work : 578
The id doesn't work : 583
The id doesn't work : 597
The id doesn't work : 667
The id doesn't work : 692
The id doesn't work : 747
The id doesn't work : 800
The id doesn't work : 826
The id doesn't work : 837
The id doesn't work : 891
The id doesn't work : 949
The id doesn't work : 957
The id doesn't work : 963
The id doesn't work : 1003
The id doesn't work : 1017
The id doesn't work : 1023
The id doesn't work : 1033
The id doesn't work : 1047
The id doesn't work : 1074
The id doesn't work : 1091


## Training

In [120]:


client.train.train_version(app_id, versionId)
waiting = True
while waiting:
    info = client.train.get_status(app_id, versionId)

    # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
    waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
    if waiting:
        print ("Waiting 10 seconds for training to complete...")
        time.sleep(10)
    else: 
        print ("trained")
        waiting = False




Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
trained


In [10]:
app_id="c4cb4b1a-f70d-4575-a4d5-b04c7e1d4812"

## Predict

In [11]:

# Mark the app as public so we can query it using any prediction endpoint.
# Note: For production scenarios, you should instead assign the app to your own LUIS prediction endpoint. See:
# https://docs.microsoft.com/en-gb/azure/cognitive-services/luis/luis-how-to-azure-subscription#assign-a-resource-to-an-app
client.apps.update_settings(app_id, is_public=True)

responseEndpointInfo = client.apps.publish(app_id, versionId, is_staging=False)


runtimeCredentials = CognitiveServicesCredentials(predictionKey)
clientRuntime = LUISRuntimeClient(endpoint=predictionEndpoint, credentials=runtimeCredentials)


# Production == slot name
predictionRequest = { "query" : "Hello i would like to book a flight from Paris to Marseille with 300$ budget" }

#result = clientRuntime.prediction.resolve(app_id, "Production", predictionRequest)
predictionResponse = clientRuntime.prediction.get_slot_prediction(app_id, "Production", predictionRequest)
print("Top intent: {}".format(predictionResponse.prediction.top_intent))
print("Sentiment: {}".format (predictionResponse.prediction.sentiment))
print("Intents: ")
#print('score:'.format(result.top_scoring_intent.intent))

for intent in predictionResponse.prediction.intents:
    print("\t{}".format (json.dumps (intent)))
print("Entities: {}".format (predictionResponse.prediction.entities))

Top intent: FlyMe
Sentiment: {'additional_properties': {}, 'label': 'positive', 'score': 0.9538003}
Intents: 
	"FlyMe"
Entities: {'FlyMeBooking': [{'or_city': ['Paris'], 'dst_city': ['Marseille'], 'budget': ['300']}], 'number': [300]}


## Scoring

In [122]:
bad_list=[1101,1106,1119,1126,1135,1140,1166,1167,1180,1190,1194,1205,1207,1208,1229,1243,1250,1254,1257,1265,1273,1275,1276,1281,1283,1285,1289,1320,1325]

In [144]:
merge=[]

for i in range(1100,1366): #20% test
    if i not in bad_list:
        merge.append(Json_Luis(dataframe.iloc[i]))


266

In [124]:
with open('test.json', 'w') as outfile:
     json.dump(merge, outfile, indent = 4,
               ensure_ascii = False)

In [182]:
print(predictionResponse.prediction.sentiment.score)

0.9538003


In [183]:
score=0
count=0
x=0
for i in range(1100,1366): #20% test


# Production == slot name
    predictionRequest = { "query" : dataframe.iloc[i].text }
    
#result = clientRuntime.prediction.resolve(app_id, "Production", predictionRequest)
    predictionResponse = clientRuntime.prediction.get_slot_prediction(app_id, "Production", predictionRequest)
    x=predictionResponse.prediction.sentiment.score
    count=count+x
score=count/266
print('Le sentiment global sur le fichier test est:',score)
    

    

Le sentiment global sur le fichier test est: 0.5612352958533835


In [176]:
score=0
count=0
x=0
for i in range(1100,1366):
    utterance = dataframe.iloc[i].text
    ##########

    # The headers to use in this REST call.
    headers = {
    }

    # The URL parameters to use in this REST call.
    params ={
        'query': utterance,
        
        'subscription-key': predictionKey
    }


    # Make the REST call.
    response = requests.get(f'{predictionEndpoint}luis/prediction/v3.0/apps/{app_id}/slots/production/predict', headers=headers, params=params)

    # Display the results on the console.
    rep=response.json()
    x=rep['prediction']['intents']['FlyMe']['score']
    count=count+x
score=count/266
print('Le score global sur le fichier test est :',score)

Le score global sur le fichier test est : 0.99972277481203
